# Generador de base para aplicar a recrédito

Este código genera una base de datos con el monto que se le puede ofertar a un cliente para que los ingresos por recrédito a n meses (con n dado y variable) sobrepasen los ingresos cancelados por recréditos y las reservas.

In [1]:
import pandas as pd
import pyodbc
from utils import ingresos_financieros, meses_entre_fechas, ingreso_fin_entre_fechas, calcular_cuota, cumple_15
from datetime import datetime, date
import numpy as np
import warnings
warnings.filterwarnings("ignore")

## Conexión a los datos

In [2]:
# conectarse a la base de datos
def es_lectura_sql():
    # Establecer conexión
    conn = pyodbc.connect('DRIVER={SQL Server};'
                        'SERVER=BTBIDB-VM;'
                        'DATABASE=LNDDW;'
                        'Trusted_Connection=yes;')

    # Conectarse a datos de la cartera
    with open ("C:/Users/mariajose_chinchilla/Desktop/Queries/BTBIDBVM/evolucion_tasas.sql", "r") as q1:
        query1 = q1.read()
    # Leer datos con pandas
    cartera = pd.read_sql_query(query1, conn) 
    # Leer datos
    return cartera

In [3]:
df = pd.read_excel(r"C:\Users\mariajose_chinchilla\Desktop\Agosto\db\cartera julio.xlsx")
df = df[df["FECHA_VENCIMIENTO"] > datetime.today()]

KeyboardInterrupt: 

In [ ]:
df["PORCENTAJE PAGADO"] = 1 - df["SALDO"] / df["MONTO DESEMBOLSADO"]

## Programación de la desigualdad y funciones útliles

In [ ]:
def desigualdad_recredito(alpha:float, ingresos_cancelados:float, tasa:float, 
                          plazo:int, n:int, cumple15: int) -> float:
    factor2 = ((1+tasa)**n - 1 - (0.05*0.45)*(1-cumple15) - (alpha / 12)*n + ((1+tasa)**plazo / ((1+tasa)**plazo - 1)) * (tasa*n + 1 - (1+tasa)**n))    
    return ingresos_cancelados * factor2**-1

## Calcular columnas importantes

Calcularemos columnas importantes para poder generar la base. Estas incluyen la cuota actual del cliente, la TPP del cliente y los ingresos financieros que el cliente representa con todos sus créditos activos a diciembre 2024.

In [ ]:
# calcular los meses transurridos
df["MESES TRANSCURRIDOS"] = df.apply(lambda x: meses_entre_fechas(x["FECHA_APERTURA"], datetime.today()), axis=1)

# meses a inicio son los meses transcurridos del crédito al mes en el que se hará la oferta
mes_inicio = 8
df["MESES A INICIO"] = df.apply(lambda x: meses_entre_fechas(x["FECHA_APERTURA"], date(2024, mes_inicio, 30)), axis=1)

df["MESES A DICIEMBRE DESDE INICIO"] = df.apply(lambda x: meses_entre_fechas(date(2024, mes_inicio, 30), date(2024, 12, 31)), axis=1)

In [ ]:
# calcular los ingresos cancelados por cliente considerando los meses que faltan
mes_inicio = "AGOSTO"
df[f"INGRESOS MES {mes_inicio} A DICIEMBRE"] = df.apply(lambda x: ingreso_fin_entre_fechas(x["MONTO DESEMBOLSADO"],
                                                                                           x["TASA"]/1200,
                                                                                             x["PLAZO"],
                                                                                             x["MESES A INICIO"],
                                                                                             x["MESES A INICIO"]  
                                                                                             + x["MESES A DICIEMBRE DESDE INICIO"]),
                                                                                             axis=1)

In [ ]:
# calcular la TPP del cliente en el banco para usar esto como tasa para el recrédito después
df["FACTOR TASA"] = df["MONTO DESEMBOLSADO"] * df["TASA"]
df["CUMPLE 15%"] = df.apply(lambda x: cumple_15(1 - x["SALDO"]/x["MONTO DESEMBOLSADO"]), axis=1)
df_agrupado = df.groupby(by="CODIGO_CLIENTE").agg(suma_factor=("FACTOR TASA", "sum"),
                                                suma_monto=("MONTO DESEMBOLSADO", "sum"),
                                                TASA_MINIMA=("TASA", "min"),
                                                TASA_MAXIMA=("TASA", "max"),
                                                DEUDA_INTERNA=("SALDO", "sum"),
                                                PLAZO_MAX=("PLAZO", "max"),
                                                CUMPLE15=("CUMPLE 15%", "min"),
                                                INGRESOS_A_CANCELAR=(f"INGRESOS MES {mes_inicio} A DICIEMBRE", "sum")).reset_index()
df = pd.merge(df, df_agrupado, on="CODIGO_CLIENTE", how="left")
# calcular tpp del cliente
df["TPP CLIENTE"] = df["suma_factor"] / df["suma_monto"]

In [ ]:
# calcular cuota de cada credito y luego la cuota total mensual del cliente
df["CUOTA MENSUAL"] = df.apply(lambda x: calcular_cuota(x["MONTO DESEMBOLSADO"],
                                                          x["TASA"]/1200,
                                                          x["PLAZO"]), axis=1)

df_agrupado = df.groupby(by="CODIGO_CLIENTE")["CUOTA MENSUAL"].sum().reset_index()
df_agrupado.columns = ["CODIGO_CLIENTE", "CUOTA TOTAL CLIENTE"]
df = pd.merge(df, df_agrupado, on="CODIGO_CLIENTE", how="left")

## Generar base

Con la informació de nuestros clientes, podemos agrupar y ver cuál es su deuda interna total, estudiar cuánto se le debería otorgar en recrédito para que el ingreso financiero sea mayor a las reservas, el ingreso cancelado por recrédito y los pasivos. Posteriormente, compararemos si este monto es suficiente para consolidarlo internamente y ofrecerle un recrédito.

In [ ]:
# generar base para opciones de recrédito
# generar base normalizada por cliente
df.rename(columns={"TASA": "TASA ACTUAL"}, inplace=True)
df_nivel_cliente = df[["CODIGO_CLIENTE", "DPI", "TASA_MINIMA", "TASA_MAXIMA",
                        "TPP CLIENTE", "CUOTA TOTAL CLIENTE", "DEUDA_INTERNA",
                        "PLAZO_MAX", "RCI_INTERNO", "CUMPLE15", "SCORE_RIESGO",
                        "TASA ACTUAL",
                        f"INGRESOS MES {mes_inicio} A DICIEMBRE"]].drop_duplicates(subset="CODIGO_CLIENTE")

tasa_elegida = "TPP CLIENTE"
alpha = (0.0747 + 0.0025) / (1 - 0.146) + 0.008
df_nivel_cliente["MONTO A OFERTAR RECREDITO"] = df.apply(lambda x: desigualdad_recredito(alpha,
                                                        x[f"INGRESOS MES {mes_inicio} A DICIEMBRE"],
                                                        0.19/1200, x["PLAZO_MAX"],
                                                        x["MESES A DICIEMBRE DESDE INICIO"],
                                                        x["CUMPLE15"]), axis=1)

df_nivel_cliente["MONTO A OFERTAR RECREDITO"] = df_nivel_cliente.apply(lambda x: min(300000, 1.75 * max(x["MONTO A OFERTAR RECREDITO"], x["DEUDA_INTERNA"])), axis=1)

df_nivel_cliente["CUOTA NUEVA"] = df_nivel_cliente.apply(lambda x: calcular_cuota(x["MONTO A OFERTAR RECREDITO"],
                                                                                x[tasa_elegida]/1200,
                                                                                x["PLAZO_MAX"]), axis=1)

In [ ]:
# solamente sacar a los que sí podrían y les interesaría el recrédito
df_nivel_cliente["MEJORA CUOTA"] = np.where(df_nivel_cliente["CUOTA TOTAL CLIENTE"] > df_nivel_cliente["CUOTA NUEVA"], 1, 0)

df_posibles = df_nivel_cliente[(df_nivel_cliente["MONTO A OFERTAR RECREDITO"] <= 500000) & (df_nivel_cliente["MEJORA CUOTA"] == 1)
                               & (df_nivel_cliente["DEUDA_INTERNA"] <= df_nivel_cliente["MONTO A OFERTAR RECREDITO"])
                               & ((df_nivel_cliente["RCI_INTERNO"].isna()) | (df_nivel_cliente["RCI_INTERNO"] < 60))
                               & (df_nivel_cliente["MONTO A OFERTAR RECREDITO"] > 0)]

df_posibles.drop("TASA_MINIMA", inplace=True, axis=1)
df_posibles.drop("TASA_MAXIMA", inplace=True, axis=1)
df_posibles.rename(columns={"MONTO A OFERTAR RECREDITO": "MONTO A DESEMBOLSAR", 
                            "CUOTA TOTAL CLIENTE": "CUOTA ACTUAL",
                            "TPP CLIENTE": "TASA",
                            "PLAZO_MAX": "PLAZO"}, inplace=True)

In [ ]:
#df_posibles.to_excel(rf"C:\Users\mariajose_chinchilla\Documents\GitHub\proyectos_varios\Salidas\comerciales\Ensyos{mes_inicio}_2.xlsx")
# df_nivel_cliente.to_excel(rf"C:\Users\mariajose_chinchilla\Documents\GitHub\proyectos_varios\Salidas\comerciales\Base todos {mes_inicio}.xlsx")

## Analizar impactos de la iniciativa

In [ ]:
#ofertas = pd.read_excel(r"C:\Users\mariajose_chinchilla\Documents\GitHub\proyectos_varios\Salidas\comerciales\Ofertas recrédito AGOSTO consolidación interna total.xlsx")
ofertas = df_posibles.copy()
ofertas.rename(columns={f"INGRESOS MES {mes_inicio} A DICIEMBRE": "INGRESOS A CANCELAR"}, inplace=True)

In [ ]:
dic_meses = {"AGOSTO": 8, "SEPTIEMBRE": 9, "OCTUBRE": 10, "NOVIEMBRE": 11, "DICIEMBRE": 12}
ofertas["RESERVAS POR REESTRUCTURA"] = (1 - ofertas["CUMPLE15"]) * (0.05 * 0.45) * ofertas["MONTO A DESEMBOLSAR"]
ofertas["PASIVOS"] = ofertas["MONTO A DESEMBOLSAR"] / 12 * (12 - dic_meses.get(mes_inicio)) * alpha
ofertas["INGRESOS A RECIBIR EN EL RESTO DEL AÑO"] = ofertas.apply(lambda x: ingreso_fin_entre_fechas(x["MONTO A DESEMBOLSAR"],
                                                                                                     x["TASA"]/1200,
                                                                                                     x["PLAZO"],
                                                                                                     dic_meses.get(mes_inicio),
                                                                                                     12), axis=1)

ofertas["MARGEN FINANCIERO"] = ofertas["INGRESOS A RECIBIR EN EL RESTO DEL AÑO"] - ofertas["RESERVAS POR REESTRUCTURA"]- ofertas["PASIVOS"] - ofertas["INGRESOS A CANCELAR"]

ofertas = ofertas[ofertas["MARGEN FINANCIERO"] > 0]

In [ ]:
#ofertas.to_excel(rf"C:\Users\mariajose_chinchilla\Documents\GitHub\proyectos_varios\Salidas\comerciales\ensayo{mes_inicio}_2.xlsx")